In [ ]:
from networkx import (Graph, draw, draw_spectral, draw_random, draw_spring, draw_shell,
                      get_node_attributes, info, floyd_warshall_numpy, all_pairs_shortest_path,
                      floyd_warshall) # pip install --user networkx
from random import randint
from sys import maxsize
from openpyxl import load_workbook # pip install --user openpyxl
from itertools import chain, islice
from collections import namedtuple
from re import match, VERBOSE, compile
import warnings
from matplotlib.pyplot import show
from functools import partial, wraps

In [ ]:
%matplotlib inline

In [ ]:
ls

In [ ]:
EXCEL_FILENAME = r'wire_journal_48_53.xlsx'

In [ ]:
SWITCH_MODEL = 'SwitchX FDR SX6025'

In [ ]:
COMPUTATIONAL_NODE_MODEL = 'Connect-IB FDR'

In [ ]:
switch_regex = compile(
    r"""
    КГК\.       # literally match what is written here
    (?P<rack>\d+)\.        # rack number is one or more digits, followed by dot
    (?P<second_number>\d+)\.            # then goes another non-negative integer followed by dot
    (?P<last_number>\d+)            # and another integer of the same form
    """,
    VERBOSE)                      

In [ ]:
RACK_PAIRS = ((48, 49), (50, 51), (52, 53))

In [ ]:
def get_column_name(column):
    """Takes column as tuple as argument and returns
    its name as string"""
    return column[0].column

In [ ]:
def extract_columns(worksheet, column_names):
    """
    parameters:
        worksheet -- worksheet
        column_names -- list of strings, for example
            ['A', 'C', 'E']
    returns:
        list of columns, where every column is represented
        as a tuple"""
    all_columns = worksheet.columns
    extracted_columns = [col for col in all_columns
                         if get_column_name(col) in column_names]
    assert len(extracted_columns) == len(column_names)
    return extracted_columns

In [ ]:
def columns_to_tuples(columns):
    """parameters:
        columns -- columns as a tuple/list of tuples
    returns:
        list of lists/tuples, each one represents a row"""
    return [[cell.value for cell in row] for row in zip(*columns)]

In [ ]:
def parse_switch_pairs(workbook):
    """Parse openpyxl workbook and extract a list of
    pairs of switches. Pair (A, B) means that swithes A
    and B are connected.
    Returns list of pairs of strings."""
    return list(chain(*[columns_to_tuples(extract_columns(worksheet, ['C', 'K']))
            for worksheet in workbook]))

In [ ]:
SwitchProperties = namedtuple('SwitchProperties', ['model', 'ports_taken'])

In [ ]:
ComputationalNodeProperties = namedtuple(
    'ComputationalNodeProperties', ['model'])

In [ ]:
EdgeProperties = namedtuple('EdgeProperties', ['material'])

In [ ]:
def get_rack(switch):
    return switch_regex.match(switch).group('rack')

In [ ]:
def get_matching_computational_nodes(switch):
    """params:
        switch -- string, name of the switch
    returns:
        list of strings which are names of computational
        nodes connected to this switch"""
    get_thingie = switch_regex.match(switch).group
    return [
        'n{0}{1}{2:02d}'.format(
            get_thingie('rack'),
            get_thingie('second_number'),
            (int(get_thingie('last_number')) - 1) * 8 + i
        )
        for i in range(1, 9)
    ]

In [ ]:
def maybe_add_switch(switch, graph):
    """If the switch is already in the graph, does nothing.
    Otherwise adds it and all the computational nodes it should have
    according to lom2 rules."""
    if switch in graph:
        return
    graph.add_node(switch, properties=SwitchProperties(
        model=SWITCH_MODEL, ports_taken=8))
    # add computational nodes connected directly to this switch
    computational_nodes = get_matching_computational_nodes(switch)
    graph.add_nodes_from(
        computational_nodes,
        properties=ComputationalNodeProperties(model=COMPUTATIONAL_NODE_MODEL))
    graph.add_edges_from(((switch, comp) for comp in computational_nodes),
                         properties=EdgeProperties(material='backplane'))

In [ ]:
def determine_material_between_switches(switch1, switch2):
    racks_of_switches = [get_rack(switch) for switch in (switch1, switch2)]
    if any(
            all(rack in rack_pair for rack in racks_of_switches)
            for rack_pair in RACK_PAIRS):
        # they are in the same pair of racks
        return 'copper'
    return 'optic'

In [ ]:
def add_connection_between_switches(switch1, switch2, graph):
    graph.add_edge(
        switch1, switch2,
        properties=EdgeProperties(
            material=determine_material_between_switches(switch1, switch2)))

In [ ]:
def create_lom2_graph(filename):
    workbook = load_workbook(filename)
    switch_connections = parse_switch_pairs(workbook)
    graph = Graph()
    for pair_of_switches in switch_connections:
        for switch in pair_of_switches:
            maybe_add_switch(switch, graph)
        add_connection_between_switches(*pair_of_switches, graph=graph)
    return graph

In [ ]:
def get_switch_nodes(graph):
    return (node for node in graph.nodes() if switch_regex.match(node) is not None)

In [ ]:
def get_non_switch_nodes(graph):
    return (node for node in graph.nodes() if switch_regex.match(node) is None)

In [ ]:
def get_switches_only(graph):
    """Returns copy of graph removing all nodes except switches"""
    copy = graph.copy()
    copy.remove_nodes_from(get_non_switch_nodes(graph))
    return copy

In [ ]:
def print_graph_properties(graph, graph_name):
    print("Graph '{0}' has {1} nodes and {2} edges".format(graph_name, len(graph), len(graph.edges())))

In [ ]:
def return_list(function):
    @wraps(function)
    def decorated(*args, **kwargs):
        return list(function(*args, **kwargs))
    return decorated

In [ ]:
def itake_n(sequence, count):
    return islice(sequence, 0, count)

In [ ]:
take_n = return_list(itake_n)

In [ ]:
def check_ugly_paths_correct_len(num_nodes, ugly_paths):
    assert len(ugly_paths) == num_nodes
    assert all(len(list_of_nodes) == num_nodes for list_of_nodes in ugly_paths.values())

In [ ]:
def get_edge_properties(graph, node1, node2):
    return graph[node1][node2]['properties']

In [ ]:
def get_node_properties(graph, node):
    return graph.node[node]['properties']

In [ ]:
def nice_shortest_paths(graph):
    ugly_paths = all_pairs_shortest_path(graph)
    check_ugly_paths_correct_len(len(graph), ugly_paths)

In [ ]:
warnings.simplefilter('ignore')
graph = create_lom2_graph(EXCEL_FILENAME)

In [ ]:
print_graph_properties(graph, 'Full')

In [ ]:
switches_only = get_switches_only(graph)

In [ ]:
print_graph_properties(switches_only, 'Switches Only')

In [ ]:
nice_shortest_paths(graph)

In [ ]:
ugly_paths = all_pairs_shortest_path(switches_only)

In [ ]:
take_n(ugly_paths, 10)

In [ ]:
take_n(ugly_paths['КГК.51.4.1'], 10)

In [ ]:
ugly_paths['КГК.51.4.1']['КГК.48.6.3']

In [ ]:
get_edge_properties(graph,'КГК.51.4.1', 'КГК.50.3.4')

In [ ]:
graph.node['КГК.51.4.1']

In [ ]:
get_node_properties(graph, 'КГК.51.4.1')